In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
device = "cuda"

In [ ]:
import PyPDF2
segments = []
def read_pdf_segments(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            segment = page.extract_text()
            segments.append(segment)
        
        return segments
read_pdf_segments("narayan-kwach.pdf")

In [ ]:
books = {
        "book1": "KarmaYoga.pdf",
        "book2": "PracticalVedanta.pdf",
        "book3": "godan.pdf",
        "book4": "abridged-autobiography-hindi.pdf",
        "book5": "narayan-kwach.pdf"
}
segmented_books = {}
for book in books:
    segmented_books[book] = read_pdf_segments(books[book])
segmented_books.keys()

In [ ]:
res = ""
resfin = []
theme = "न्यास"
for segment in segments:
  messages = [
    {
      "role": "user", "content":'''I will provide you with some text and a theme, your job is to identify the language of the provided text and theme, then generate a question based upon the theme in English. 
                                   Make sure the question you generate is answerable only by the provided text.
                                   Please make no assumptions, and answer in English only.
                                   In the case where no such question is possible, answer with the word NO only.''',
    },
    {
      "role": "assistant", "content":"Sure, please provide me with the text and the theme."
    },

    {
      "role": "user", "content":"The text is demarkated by triple backticks:```होरी एक गरीब किसान था, जो मुश्किल से गुजारा कर पा रहा था। अपनी कठिनाइयों के बावजूद, वह एक गाय खरीदने का सपना देखता था, जो गाँव में समृद्धि का प्रतीक थी।```. The theme is demarkated by double backticks:``होरी की गाय खरीदने की इच्छा``.",
    },

    {"role":"assistant","content": "Why did Hori want a cow?"
    },
    {
      "role":"user","content": f'''The text is demarkated by triple backticks:```{segment}```. 
                                   The theme is demarkated by double backticks:``{theme}``.''',
    }
  ] 

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  model.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  res = decoded[0]
  
  last_inst_index = res.rfind('[/INST]')
  if last_inst_index != -1:
      res = res[last_inst_index + len('[/INST]'):]
    
    
  question_mark_index = res.find('?')
  if question_mark_index != -1:
      res = res[:question_mark_index + 1]  
  
  if "NO" not in res.strip():
    resfin.append(res)
  if(len(resfin) == 3):
    break
print(resfin)

In [ ]:
# Load the NLLB model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer_ts = AutoTokenizer.from_pretrained(model_name)
model_ts = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:

# Text to translate
for res in resfin:
    text = res

    # Tokenize the input text for the source language (English)
    tokenizer.src_lang = "eng_Latn"
    inputs = tokenizer_ts(text, return_tensors="pt")

    # Set the target language (Sanskrit) by using the corresponding token ID
    target_lang = "san_Deva"
    forced_bos_token_id = tokenizer_ts.convert_tokens_to_ids(target_lang)

    # Generate the translation with a limit on the number of new tokens generated
    translated_tokens = model_ts.generate(**inputs, forced_bos_token_id=forced_bos_token_id, max_new_tokens=50)

    # Decode the translated tokens to get the final translated text
    translated_text = tokenizer_ts.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    # Print the translated text
    print(translated_text)
    print("\n")
